In [1]:
pip install praw pandas matplotlib wordcloud nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pytesseract pillow langdetect

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR"

In [1]:
!pwd                # shows your current folder
!git status         # check uncommitted changes
!git add .
!git commit -m "Scraping post and comment data"
!git push origin main

'pwd' is not recognized as an internal or external command,
operable program or batch file.


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


[main 1cfd0b8] Scraping post and comment data
 1 file changed, 7 insertions(+), 12 deletions(-)


To https://github.com/Starlight0901/Reddit_SriLanka_Social_Insight.git
   090cabf..1cfd0b8  main -> main


### Reddit API checking

In [2]:
import praw

reddit = praw.Reddit(
    client_id="3VypVlHsKWYOd43a4XZ1Qw",            # your client ID
    client_secret="RnwapkXWjkuee9Rv8B5rEoaOscL4wg", # your secret
    user_agent="NLP_SriLankaResearchApp by u/Cute_Firefighter1926"
)


In [3]:
print(reddit.user.me())   # should print "None" (since it's read-only)
for post in reddit.subreddit("srilanka").hot(limit=5):
    print(post.title)

None
Why on earth did Daily Mirror take this post down?
Double check before you buy from Abans 11.11 sale
Guess where this video is from.
Here we go. People opposing the implementation of Sex Education. How can we ever improve as a country with this mindset? What do you guys think?
Is Tech the only industry offering good pay in SL ?


# Pipeline

In [31]:
import os
import praw
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
import logging
import time
import matplotlib as plt
from wordcloud import WordCloud

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tharu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
subreddit_name = "srilanka"
sub = reddit.subreddit(subreddit_name)

## Pipeline to collect data from Posts

### Important Functions

In [7]:
# Function to make sure the sentence is meaningful (words more than 4 words)

def valid_text(text):
    """Check if text exists and has >4 words."""
    return isinstance(text, str) and len(text.split()) > 4

In [8]:
# Function to collect all the posts from "new"

def collect_posts_from_new():
    """Collect all available latest posts from the subreddit"""
    data = []
    print("Collecting all available latest posts...")

    for post in sub.new(limit=None):
        content = f"{post.title} {post.selftext}".strip()
        if valid_text(content):
            data.append({
                "type": "post",
                "source": "new",
                "keyword": None,
                "id": post.id,
                "created_utc": post.created_utc,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,  # ✅ Save subreddit name
                "content": content,
                "score": post.score,
                "num_comments": post.num_comments
            })

    print(f"Collected {len(data)} posts.")
    return data

In [9]:
# Collect posts from "hot"

def collect_posts_from_hot(limit=None):
    """Collect all available hot posts from the subreddit"""
    data = []
    print("Collecting all hot posts...")

    for post in sub.hot(limit=limit):
        content = f"{post.title} {post.selftext}".strip()
        if valid_text(content):
            data.append({
                "type": "post",
                "source": "hot",
                "keyword": None,
                "id": post.id,
                "created_utc": post.created_utc,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,
                "content": content,
                "score": post.score,
                "num_comments": post.num_comments
            })

    print(f"Collected {len(data)} hot posts.")
    return data

In [10]:
# Collect posts from "top"

def collect_posts_from_top(limit=None, time_filter="all"):
    """Collect top posts from the subreddit with optional time_filter"""
    data = []
    print(f"Collecting top posts (time_filter='{time_filter}')...")

    for post in sub.top(limit=limit, time_filter=time_filter):
        content = f"{post.title} {post.selftext}".strip()
        if valid_text(content):
            data.append({
                "type": "post",
                "source": "top",
                "keyword": None,
                "id": post.id,
                "created_utc": post.created_utc,
                "author": str(post.author),
                "subreddit": post.subreddit.display_name,
                "content": content,
                "score": post.score,
                "num_comments": post.num_comments
            })

    print(f"Collected {len(data)} top posts.")
    return data


In [11]:
# Function to collect all the posts from search for each keyword

def collect_posts_from_search(keywords):
    """Collect posts via keyword search"""
    data = []
    for kw in keywords:
        print(f"🔎 Searching for keyword: {kw}")
        try:
            for post in sub.search(kw, sort="new", time_filter="all", limit=None):
                content = f"{post.title} {post.selftext}".strip()
                if valid_text(content):
                    data.append({
                        "type": "post",
                        "source": "search",
                        "keyword": kw,
                        "id": post.id,
                        "created_utc": post.created_utc,
                        "author": str(post.author),
                        "subreddit": post.subreddit.display_name,  # ✅ Save subreddit name
                        "content": content,
                        "score": post.score,
                        "num_comments": post.num_comments
                    })
            print(f"✅ Done with keyword '{kw}', total so far: {len(data)}")
        except Exception as e:
            print(f"⚠️ Error with keyword '{kw}': {e}")
        time.sleep(2)  # avoid rate limits
    return data

### Collecting Data

In [12]:
# Collect from .new() 

posts_new = collect_posts_from_new()

Collected 997 posts.


In [13]:
# Collect from .hot() 

posts_hot = collect_posts_from_hot()

Collected 998 hot posts.


In [14]:
# Collect from .top() 

posts_top = collect_posts_from_top()

Collected 231 top posts.


In [15]:
# Collect from .search() using "Keywords"

keywords = [
    # --- Politics and Governance ---
    "politics", "government", "parliament", "president", "prime minister", "minister", "election", 
    "voting", "policy", "corruption", "protest", "democracy", "dictatorship", "rights", 
    "foreign relations", "IMF", "debt", "reform", "budget cuts", "privatization", "diplomacy",
    "public administration", "bureaucracy", "governance", "development plan",

    # --- Economy and Employment ---
    "economy", "inflation", "finance", "budget", "tax", "trade", "business", "industry",
    "investment", "employment", "unemployment", "job", "market", "entrepreneurship", "startups", 
    "SMEs", "cost of living", "poverty", "income", "wages", "labor", "productivity",

    # --- Education and Learning ---
    "education", "school", "preschool", "primary education", "secondary education", "college", 
    "university", "campus", "faculty", "higher education", "studies", "student", "students", 
    "teacher", "lecturer", "professor", "principal", "tuition", "scholarship", "exam", "O/L", "A/L",
    "curriculum", "syllabus", "textbook", "subject", "education reform", "education policy", 
    "private school", "public school", "international school", "ministry of education", 
    "education inequality", "literacy", "dropout", "school fees", "admission", 
    "online learning", "distance learning", "e-learning", "virtual classroom", 
    "digital education", "EdTech", "learning app", "AI in education", "STEM", "ICT", 
    "vocational training", "technical education", "TVET", "teacher training", 
    "exam results", "Z-score", "university admission", "research", "academic", "publication", 
    "school reopening", "school closure", "classroom", "homework", "discipline", "student union", 
    "education budget", "education crisis", "education rights",

    # --- Health and Well-being ---
    "health", "hospital", "doctor", "nurse", "medicine", "pharmacy", "covid", 
    "mental health", "nutrition", "vaccination", "public health", "epidemic", "wellness",

    # --- Transport and Infrastructure ---
    "transport", "bus", "train", "traffic", "road", "airport", "port", "highway", 
    "vehicle", "fuel", "petrol", "transportation policy", "traffic jam",

    # --- Tourism and Culture ---
    "tourism", "travel", "visa", "hotel", "heritage", "beach", "wildlife", "culture", 
    "festival", "tradition", "food", "art", "music", "dance", "religion", "temple", "church", "mosque",

    # --- Society and Community ---
    "society", "community", "religion", "ethnicity", "language", "gender", "family", 
    "youth", "marriage", "divorce", "social media", "influence", "violence", "addiction",

    # --- Technology and Innovation ---
    "technology", "internet", "mobile", "AI", "machine learning", "robotics", 
    "startup", "innovation", "science", "engineering", "research", "data science", 
    "cybersecurity", "software", "coding", "programming", "app", "digital divide",

    # --- Environment and Agriculture ---
    "environment", "climate", "pollution", "forest", "wildlife", "sustainability", 
    "agriculture", "farming", "paddy", "organic farming", "flood", "drought", "waste", 
    "recycling", "deforestation",

    # --- Sports and Recreation ---
    "sports", "cricket", "football", "rugby", "athletics", "games", "Olympics", 
    "tournament", "fitness", "training", "coach", "sports development",

    # --- Development and Infrastructure ---
    "development", "infrastructure", "housing", "urban", "construction", "energy", 
    "electricity", "renewable energy", "water", "sanitation", "drainage", "smart city",

    # --- Law and Order ---
    "law", "justice", "court", "crime", "police", "rights", "freedom", 
    "protest", "migration", "refugee", "diaspora", "human rights",

    # --- Media and Communication ---
    "media", "journalism", "press", "television", "cinema", "radio", 
    "music", "art", "literature", "publication", "podcast",
    "communication", "telecom", "Dialog", "SLT", "internet speed", 
    "data", "mobile network", "social network", "broadcast",

    # --- Defense and Security ---
    "military", "security", "terrorism", "war", "LTTE", "peace", 
    "national security", "army", "navy", "air force",

    # --- Society ---
    "entertainment", "movies", "gaming", "memes", "news", "update", "local event", 
    "weather", "rain", "heatwave", "temperature", "food prices", 
    "charity", "donation", "public service", "volunteer", "NGO", "social work"
]

posts_search = collect_posts_from_search(keywords)

🔎 Searching for keyword: politics
✅ Done with keyword 'politics', total so far: 228
🔎 Searching for keyword: government
✅ Done with keyword 'government', total so far: 475
🔎 Searching for keyword: parliament
✅ Done with keyword 'parliament', total so far: 696
🔎 Searching for keyword: president
✅ Done with keyword 'president', total so far: 930
🔎 Searching for keyword: prime minister
✅ Done with keyword 'prime minister', total so far: 1039
🔎 Searching for keyword: minister
✅ Done with keyword 'minister', total so far: 1264
🔎 Searching for keyword: election
✅ Done with keyword 'election', total so far: 1492
🔎 Searching for keyword: voting
✅ Done with keyword 'voting', total so far: 1719
🔎 Searching for keyword: policy
✅ Done with keyword 'policy', total so far: 1954
🔎 Searching for keyword: corruption
✅ Done with keyword 'corruption', total so far: 2172
🔎 Searching for keyword: protest
✅ Done with keyword 'protest', total so far: 2387
🔎 Searching for keyword: democracy
✅ Done with keywor

In [18]:
# Combine post datasets 

all_posts = posts_new + posts_hot + posts_top + posts_search
df_posts = pd.DataFrame(all_posts)

In [19]:
print("Total posts collected (without handling duplicates): ", len(df_posts))
df_posts.head(10)

Total posts collected (without handling duplicates):  43655


,type,source,keyword,id,created_utc,author,subreddit,content,score,num_comments
0,post,new,None,1otaemb,1.762771e+09,Cookiehere6969,srilanka,Is this a Scam or good investment? Haritha Lan...,2,1
1,post,new,None,1otaam5,1.762770e+09,oshan789,srilanka,Villa units for sale in Unawatuna Sri Lanka ! ...,3,0
2,post,new,None,1ot9w1v,1.762769e+09,mgssjjsks,srilanka,Whats your hot take on Sri Lanka as the title ...,3,8
3,post,new,None,1ot9kwe,1.762768e+09,Critical_Rise_exe,srilanka,Is the rs.11 deals real in Daraz?,1,3
4,post,new,None,1ot9h2f,1.762767e+09,No-Leave8971,srilanka,Need advice from the experts 🙏 [](https://www....,2,0
5,post,new,None,1ot9dyw,1.762767e+09,hotstar10,srilanka,Confusion Over Paddock Club Nugegoda’s Halal S...,0,4
6,post,new,None,1ot9da2,1.762767e+09,prav_u,srilanka,Tour to Kanneliya Rain Forest I’m planning a g...,1,1
7,post,new,None,1ot8r8a,1.762764e+09,Unusual-Witness-7304,srilanka,"ChatGPT vs electricians, my house wiring is no...",6,5
8,post,new,None,1ot8lir,1.762764e+09,negative-impactr8888,srilanka,SLTMobitel changed the superuser router passwo...,3,0
9,post,new,None,1ot8l95,1.762764e+09,Jakesbond007,srilanka,Can we hand carry medicine from abroad into Sr...,1,3


In [20]:
df_posts.drop_duplicates(subset=["content"], inplace=True)
print(f"Total posts collected (unique): {len(df_posts)}")

Total posts collected (unique): 18840


In [21]:
# Save df_posts to a CSV file
df_posts.to_csv("df_posts.csv", index=False)  

## Pipeline to collect data from Comments

In [22]:
# Function to collect first 20 comments for up to 6000 posts

def collect_comments_from_posts(post_ids, max_posts=10000, max_comments_per_post=50):
    """Fetch the first N comments for up to max_posts post IDs"""
    comments_data = []
    counter = 0

    # Limit the number of posts to max_posts
    post_ids = post_ids[:max_posts]
    print(f"Collecting comments for {len(post_ids)} posts (up to {max_comments_per_post} comments each)...")

    for pid in post_ids:
        counter += 1

        try:
            submission = reddit.submission(id=pid)
            # Fetch only the top-level comments (no "load more")
            submission.comments.replace_more(limit=0)  
            comments_list = submission.comments.list()[:max_comments_per_post]  # first 30 comments

            for comment in comments_list:
                if valid_text(comment.body):
                    comments_data.append({
                        "type": "comment",
                        "source": "comment",
                        "keyword": None,
                        "id": comment.id,
                        "parent_post": pid,
                        "created_utc": comment.created_utc,
                        "author": str(comment.author),
                        "subreddit": comment.subreddit.display_name,  
                        "content": comment.body,
                        "score": comment.score,
                        "num_comments": None
                    })

            if counter % 100 == 0:
                print(f"Processed {counter} posts... total comments so far: {len(comments_data)}")

            time.sleep(2)  # avoid rate limiting

        except Exception as e:
            print(f"⚠️ Error fetching comments for {pid}: {e}")
            time.sleep(5)

    df_comments = pd.DataFrame(comments_data)
    print(f"💬 Total comments collected: {len(df_comments)}")
    return comments_data

In [23]:
# Collect comments 

comments = collect_comments_from_posts(df_posts["id"])

✅ Processed 100 posts... total comments so far: 593
✅ Processed 200 posts... total comments so far: 1311
✅ Processed 300 posts... total comments so far: 2048
✅ Processed 400 posts... total comments so far: 2759
✅ Processed 500 posts... total comments so far: 3412
✅ Processed 600 posts... total comments so far: 4105
✅ Processed 700 posts... total comments so far: 4824
✅ Processed 800 posts... total comments so far: 5705
✅ Processed 900 posts... total comments so far: 6370
✅ Processed 1000 posts... total comments so far: 7119
✅ Processed 1100 posts... total comments so far: 8634
✅ Processed 1200 posts... total comments so far: 10152
✅ Processed 1300 posts... total comments so far: 11403
✅ Processed 1400 posts... total comments so far: 12726
✅ Processed 1500 posts... total comments so far: 13786
✅ Processed 1600 posts... total comments so far: 14841
✅ Processed 1700 posts... total comments so far: 15957
✅ Processed 1800 posts... total comments so far: 16959
✅ Processed 1900 posts... total

In [24]:
df_comments = pd.DataFrame(comments)
len(df_comments)

54042

## Combine Posts and Comments

In [25]:
df_text = pd.concat([df_posts, df_comments], ignore_index=True)
df_text.drop_duplicates(subset=["content"], inplace=True)

In [28]:
df_text.head(10)

,type,source,keyword,id,created_utc,author,subreddit,content,score,num_comments,parent_post
0,post,new,None,1otaemb,1.762771e+09,Cookiehere6969,srilanka,Is this a Scam or good investment? Haritha Lan...,2,1,NaN
1,post,new,None,1otaam5,1.762770e+09,oshan789,srilanka,Villa units for sale in Unawatuna Sri Lanka ! ...,3,0,NaN
2,post,new,None,1ot9w1v,1.762769e+09,mgssjjsks,srilanka,Whats your hot take on Sri Lanka as the title ...,3,8,NaN
3,post,new,None,1ot9kwe,1.762768e+09,Critical_Rise_exe,srilanka,Is the rs.11 deals real in Daraz?,1,3,NaN
4,post,new,None,1ot9h2f,1.762767e+09,No-Leave8971,srilanka,Need advice from the experts 🙏 [](https://www....,2,0,NaN
5,post,new,None,1ot9dyw,1.762767e+09,hotstar10,srilanka,Confusion Over Paddock Club Nugegoda’s Halal S...,0,4,NaN
6,post,new,None,1ot9da2,1.762767e+09,prav_u,srilanka,Tour to Kanneliya Rain Forest I’m planning a g...,1,1,NaN
7,post,new,None,1ot8r8a,1.762764e+09,Unusual-Witness-7304,srilanka,"ChatGPT vs electricians, my house wiring is no...",6,5,NaN
8,post,new,None,1ot8lir,1.762764e+09,negative-impactr8888,srilanka,SLTMobitel changed the superuser router passwo...,3,0,NaN
9,post,new,None,1ot8l95,1.762764e+09,Jakesbond007,srilanka,Can we hand carry medicine from abroad into Sr...,1,3,NaN


In [29]:
len(df_text)

70853

In [34]:
df_text.to_csv("r_srilanka_text_dataset.csv", index=False)
print(f"Final text dataset saved — {len(df_text)} rows total!")

Final text dataset saved — 70853 rows total!


## Pipeline to extract from non-textual data (image posts)

# Exploratory Data Analysis (EDA)

In [32]:
text_blob = " ".join(df_text['content'].dropna().astype(str).tolist())

# --- Generate the word cloud ---
wc = WordCloud(
    width=1200,
    height=600,
    background_color='white',
    stopwords=set(stopwords.words('english')),
    max_words=300,
    collocations=False
).generate(text_blob)

# --- Display ---
plt.figure(figsize=(14, 7))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud of Posts and Comments from r/srilanka", fontsize=18)
plt.show()

TypeError: 'module' object is not callable